In [2]:
#   x   x   x   x   x   x   x   x   x
#   x   x           x           x   x
#   x       x       x       x       x
#   x           x   x   x           x
#   x   x   x   x   x   x   x   x   x
#   x           x   x   x           x
#   x       x       x       x       x
#   x   x           x           x   x
#   x   x   x   x   x   x   x   x   x

from model.Case import TYPE_CASE, Case

ROWS = 9
COLS = 9

themes = [('🟥', 1), ('🟦', 2), ('🟨', 3), ('🟪', 4), ('🟩', 5), ('🟧', 6)]
camemberts = [('🍓', 1), ('💙', 2), ('💛', 3), ('💜', 4), ('💚', 5), ('🧡', 6)]
cases = []   

def setup():
    i = 0
    for row in range(ROWS):
        rowItems = []
        if i == len(themes) - 1:
            i = 0
            
        for col in range(COLS):
            if ((col == row == 0) or
                (col == COLS // 2 and row == 0) or
                (col == COLS - 1 and row == 0) or  
                
                (row == ROWS - 1 and col == 0) or
                (row == ROWS - 1 and col == COLS // 2) or
                (row == ROWS - 1 and col == COLS - 1)):
                    item = camemberts.pop()
                    c = Case(position=(col, row), type_case=TYPE_CASE['theme'], theme=item[1], graf=item[0])
                    rowItems.append(c)
            elif col == 0: # Première colonne
                rowItems.append(Case((col, row), TYPE_CASE['theme'], themes[i][1], themes[i][0]))
            elif row == 0 or row == ROWS // 2: # Première ligne et ligne du milieu
                rowItems.append(Case((col, row), TYPE_CASE['theme'], themes[i][1], themes[i][0]))
            elif row == ROWS - 1: # Dernière ligne
                rowItems.append(Case((col, row), TYPE_CASE['theme'], themes[i][1], themes[i][0]))
            elif col == COLS - 1 or col == COLS // 2: # Dernière colonne et colonne du milieu
                rowItems.append(Case((col, row), TYPE_CASE['theme'], themes[i][1], themes[i][0]))
            elif col == row or (col + row) + 1 == ROWS: # Diagonales
                rowItems.append(Case((col, row), TYPE_CASE['theme'], themes[i][1], themes[i][0]))
            else:
                rowItems.append(Case((col, row), TYPE_CASE['null'])) 
                
            i += 1
            if i == len(themes) - 1:
                i = 0
                
        cases.append(rowItems)
        
    # for row_idx in range(len(cases) - 1):
    #     for col_idx in range(len(row) - 1):
    #         if col_idx > 0 and row_idx > 0:
                
            
def find_walkable_cases(position, walkable_cases = [], next_position = (0, 1)):
    row = cases[position[next_position[0]]]
    col = cases[position[next_position[1]]]
    next_case = next_position if next_position is not None else cases[row][col]
    if next_case.theme < 4:
        walkable_cases.append(next_case)
    
    find_walkable_cases(position, walkable_cases, )

def render():
    setup()
    graf = ''
    for row in cases:
        for item in row:
            graf = f'{graf}{item.graf}'
        graf = f'{graf}\r\n'
    
    print(graf)    
render()

🧡🟦🟨🟪💚🟥🟦🟨💜
🟩🟥󠁛󠁛󠁛⬛󠁛󠁛󠁛⬛🟪󠁛󠁛󠁛⬛󠁛󠁛󠁛⬛🟦🟨
🟪󠁛󠁛󠁛⬛🟥󠁛󠁛󠁛⬛🟨󠁛󠁛󠁛⬛🟩󠁛󠁛󠁛⬛🟦
🟨󠁛󠁛󠁛⬛󠁛󠁛󠁛⬛🟥🟦🟨󠁛󠁛󠁛⬛󠁛󠁛󠁛⬛🟥
🟦🟨🟪🟩🟥🟦🟨🟪🟩
🟥󠁛󠁛󠁛⬛󠁛󠁛󠁛⬛🟪🟩🟥󠁛󠁛󠁛⬛󠁛󠁛󠁛⬛🟪
🟩󠁛󠁛󠁛⬛🟦󠁛󠁛󠁛⬛🟪󠁛󠁛󠁛⬛🟥󠁛󠁛󠁛⬛🟨
🟪🟩󠁛󠁛󠁛⬛󠁛󠁛󠁛⬛🟨󠁛󠁛󠁛⬛󠁛󠁛󠁛⬛🟥🟦
💛🟪🟩🟥💙🟨🟪🟩🍓

